## Genrate one of the magic features

#### WHAT???

The question id (qid) is actually not a random given id, it is time dependent. What's more, Quora has less and less duplicated question pairs as the time pass. These phenomenons makes the qid becoming a very strong feature.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict

train_orig =  pd.read_csv('../dataset/quora-question-pairs/train.csv', header=0)
test_orig =  pd.read_csv('../dataset/quora-question-pairs/test.csv', header=0)

# "id","qid1","qid2","question1","question2","is_duplicate"
df_id1 = train_orig[["qid1", "question1"]].drop_duplicates(keep="first").copy().reset_index(drop=True)
df_id2 = train_orig[["qid2", "question2"]].drop_duplicates(keep="first").copy().reset_index(drop=True)

df_id1.columns = ["qid", "question"]
df_id2.columns = ["qid", "question"]

print(df_id1.shape, df_id2.shape)

df_id = pd.concat([df_id1, df_id2]).drop_duplicates(keep="first").reset_index(drop=True)
print(df_id1.shape, df_id2.shape, df_id.shape)

import csv
dict_questions = df_id.set_index('question').to_dict()
dict_questions = dict_questions["qid"]

new_id = 538000 # df_id["qid"].max() ==> 537933

def get_id(question):
    global dict_questions 
    global new_id 
    
    if question in dict_questions:
        return dict_questions[question]
    else:
        new_id += 1
        dict_questions[question] = new_id
        return new_id
    
rows = []
max_lines = 10
if True:
    with open('../dataset/quora-question-pairs/test.csv', 'r', encoding="utf8") as infile:
        reader = csv.reader(infile, delimiter=",")
        header = next(reader)
        header.append('qid1')
        header.append('qid2')
        
        if True:
            print(header)
            pos, max_lines = 0, 10*1000*1000
            for row in reader:
                # "test_id","question1","question2"
                question1 = row[1]
                question2 = row[2]

                qid1 = get_id(question1)
                qid2 = get_id(question2)
                row.append(qid1)
                row.append(qid2)

                pos += 1
                if pos >= max_lines:
                    break
                rows.append(row)

(290654, 2) (299364, 2)
(290654, 2) (299364, 2) (537933, 2)
['test_id', 'question1', 'question2', 'qid1', 'qid2']


In [4]:
rows[0]

['0',
 'How does the Surface Pro himself 4 compare with iPad Pro?',
 'Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?',
 538001,
 538002]

In [12]:
train_qid1 = train_orig['qid1'].as_matrix()
train_qid2 = train_orig['qid2'].as_matrix()
df = pd.DataFrame(np.array([train_qid1,train_qid2]).reshape(-1,2), columns=['magic_qid1','magic_qid2'])
df.to_csv('./features_from_model/train/magic_v25_qid.csv', index=False)

test_qid1 = [row[3] for row in rows]
test_qid2 = [row[4] for row in rows]
df = pd.DataFrame(np.array([test_qid1,test_qid2]).reshape(-1,2), columns=['magic_qid1','magic_qid2'])
df.to_csv('./features_from_model/test/magic_v25_qid.csv', index=False)